In [2]:
import pandas as pd
import numpy as np
import sqlite3
import datetime
df = pd.read_csv("201804.csv", names=('date', 'code', 'market', 'open', 'high', 'low', 'close', 'volume','is_yesterday', 'is_tomorrow'))
df.head()

,date,code,market,open,high,low,close,volume,is_yesterday,is_tomorrow
0,20180402,0101,i,21441.0,21597.0,21388.0,21388.0,956730000,NaN,NaN
1,20180402,0201,i,1716.0,1723.0,1708.0,1708.0,956730000,NaN,NaN
2,20180402,1301,t1,3800.0,3800.0,3740.0,3750.0,28000,NaN,NaN
3,20180402,1305,t1,1809.0,1817.0,1801.0,1801.0,73880,NaN,NaN
4,20180402,1306,t1,1785.0,1792.0,1776.0,1777.0,1883230,NaN,NaN


In [4]:
dbname = "market.db"
conn = sqlite3.connect(dbname)
c = conn.cursor()

In [ ]:
df.to_sql("market", conn, if_exists="replace")

In [14]:
c2 = conn.cursor()

for row in c.execute(select_sql):
    d = datetime.datetime.strptime(str(row[1]), '%Y%m%d')
    d -= datetime.timedelta(days = 1)
    date_yesterday = int(d.strftime('%Y%m%d'))
    d = datetime.datetime.strptime(str(row[1]), '%Y%m%d')
    d += datetime.timedelta(days = 1)
    date_tomorrow = int(d.strftime('%Y%m%d'))
    
    is_yesterday_sql = '''SELECT COUNT(*) FROM market WHERE date={} AND code='{}' AND market='{}\''''.format(date_yesterday, row[2], row[3])
    is_yesterday = c2.execute(is_yesterday_sql).fetchone()[0];
    update_sql = '''UPDATE market SET is_yesterday={} WHERE date={} AND code='{}' AND market='{}\''''.format(is_yesterday,row[1], row[2], row[3])
    c2.execute(update_sql);
    
    is_tomorrow_sql = '''SELECT COUNT(*) FROM market WHERE date={} AND code='{}' AND market='{}\''''.format(date_tomorrow, row[2], row[3])
    is_tomorrow = c2.execute(is_tomorrow_sql).fetchone()[0];
    update_sql = '''UPDATE market SET is_tomorrow={} WHERE date={} AND code='{}' AND market='{}\''''.format(is_tomorrow,row[1], row[2], row[3])
    c2.execute(update_sql);

conn.commit()

In [5]:
select_sql = 'select * from market'
df = pd.read_sql_query(select_sql, conn)
df.head()

,index,date,code,market,open,high,low,close,volume,is_yesterday,is_tomorrow
0,0,20180402,0101,i,21441.0,21597.0,21388.0,21388.0,956730000,0.0,1.0
1,1,20180402,0201,i,1716.0,1723.0,1708.0,1708.0,956730000,0.0,1.0
2,2,20180402,1301,t1,3800.0,3800.0,3740.0,3750.0,28000,0.0,1.0
3,3,20180402,1305,t1,1809.0,1817.0,1801.0,1801.0,73880,0.0,1.0
4,4,20180402,1306,t1,1785.0,1792.0,1776.0,1777.0,1883230,0.0,1.0


In [6]:
df_t1 = df[df['market'] == 't1']
df_t1.head()

,index,date,code,market,open,high,low,close,volume,is_yesterday,is_tomorrow
2,2,20180402,1301,t1,3800.0,3800.0,3740.0,3750.0,28000,0.0,1.0
3,3,20180402,1305,t1,1809.0,1817.0,1801.0,1801.0,73880,0.0,1.0
4,4,20180402,1306,t1,1785.0,1792.0,1776.0,1777.0,1883230,0.0,1.0
5,5,20180402,1308,t1,1768.0,1775.0,1759.0,1759.0,84700,0.0,1.0
6,6,20180402,1309,t1,32150.0,32300.0,31800.0,32250.0,57,0.0,1.0


In [16]:
df_now = df_t1[df_t1['close'] >= 100]

UniqueCodes = df_now.code.unique()
df_codes = {elem : pd.DataFrame for elem in UniqueCodes}
for key in df_codes.keys():
    df_codes[key] = df_now[:][df_now.code == key].reset_index()

,level_0,index,date,code,market,open,high,low,close,volume,is_yesterday,is_tomorrow
0,8,8,20180402,1311,t1,768.0,777.0,768.0,772.0,260,0.0,1.0
1,4224,4224,20180403,1311,t1,780.0,780.0,761.0,764.0,4920,1.0,1.0
2,8441,8441,20180404,1311,t1,765.0,768.0,763.0,764.0,550,1.0,1.0
3,12659,12659,20180405,1311,t1,774.0,779.0,768.0,774.0,2150,1.0,1.0
4,16877,16877,20180406,1311,t1,774.0,776.0,767.0,767.0,3560,1.0,0.0


In [17]:
df_codes['1311']

,level_0,index,date,code,market,open,high,low,close,volume,is_yesterday,is_tomorrow
0,8,8,20180402,1311,t1,768.0,777.0,768.0,772.0,260,0.0,1.0
1,4224,4224,20180403,1311,t1,780.0,780.0,761.0,764.0,4920,1.0,1.0
2,8441,8441,20180404,1311,t1,765.0,768.0,763.0,764.0,550,1.0,1.0
3,12659,12659,20180405,1311,t1,774.0,779.0,768.0,774.0,2150,1.0,1.0
4,16877,16877,20180406,1311,t1,774.0,776.0,767.0,767.0,3560,1.0,0.0


In [ ]:
'''
close >= 100
volume/10day >= 100000000
株価位置(200日)が40%以上
前日終値より5%以上
当日陽線(2.5%)

'''

In [61]:
UniqueDates = df_now.date.unique()
print(UniqueDates)

[20180402 20180403 20180404 20180405 20180406]


In [90]:
#現在の日付のindex
def get_now_date(UniqueDates, date):
    now_date = int(np.where(UniqueDates == date)[0])
    return now_date

In [116]:
#出来高平均
def get_mean_volume(code, now_date, span):
    first = now_date - (span - 1)
    return df_codes[code].iloc[first : first + span, :]['volume'].mean()

In [117]:
#株価位置
def get_market_position(code, now_date, span):
    first = now_date - (span - 1)
    df_span = df_codes[code].iloc[first : first + span, :]
    df_now = df_codes[code][now_date:now_date+1]
    min = df_span['low'].min()
    max = df_span['high'].max()
    nowprice = float(df_now['close'])
    return (nowprice - min) / (max - min) * 100

In [120]:
now_date = get_now_date(UniqueDates, 20180406)
print(get_mean_volume('1311', now_date, 2))
print(get_market_position('1311', now_date, 2))

2855.0
0.0


In [136]:
BuyEntry = []
SellEntry = []
BuyExit = []
SellExit = []
for date in UniqueDates:
    now_date = get_now_date(UniqueDates, date)
    mean_volume = get_mean_volume('1311', now_date, 2)
    market_position = get_market_position('1311', now_date, 2)
    print(date)
    print(mean_volume)
    print(market_position)
    
    BuyEntry.append(0)
    BuyExit.append(0)
    
    if mean_volume > 2000 and market_position > 10:
        SellEntry.append(1)
    else:
        SellEntry.append(0)
        
    if market_position > 60:
        SellExit.append(1)
    else:
        SellExit.append(0)
        
print(SellEntry)
print(SellExit)

20180402
nan
nan
20180403
2590.0
15.7894736842
20180404
2735.0
15.7894736842
20180405
1350.0
68.75
20180406
2855.0
0.0
[0, 1, 1, 0, 0]
[0, 0, 0, 1, 0]


In [139]:
def Backtest(ohlc, BuyEntry, SellEntry, BuyExit, SellExit, lots=100):
    Open = ohlc['open'].values #始値
    Lots = lots*1
    N = len(ohlc) #FXデータのサイズ
    BuyExit[N-2] = SellExit[N-2] = True #最後に強制エグジット
    BuyPrice = SellPrice = 0.0 # 売買価格

    LongTrade = np.zeros(N) # 買いトレード情報
    ShortTrade = np.zeros(N) # 売りトレード情報

    LongPL = np.zeros(N) # 買いポジションの損益
    ShortPL = np.zeros(N) # 売りポジションの損益

    for i in range(1,N):
        if BuyEntry[i-1] and BuyPrice == 0: #買いエントリーシグナル
            BuyPrice = Open[i]
            LongTrade[i] = BuyPrice #買いポジションオープン
        elif BuyExit[i-1] and BuyPrice != 0: #買いエグジットシグナル
            ClosePrice = Open[i]
            LongTrade[i] = -ClosePrice #買いポジションクローズ
            LongPL[i] = (ClosePrice-BuyPrice)*Lots #損益確定
            BuyPrice = 0

        if SellEntry[i-1] and SellPrice == 0: #売りエントリーシグナル
            SellPrice = Open[i]
            ShortTrade[i] = SellPrice #売りポジションオープン
        elif SellExit[i-1] and SellPrice != 0: #売りエグジットシグナル
            ClosePrice = Open[i]
            ShortTrade[i] = -ClosePrice #売りポジションクローズ
            ShortPL[i] = (SellPrice-ClosePrice)*Lots #損益確定
            SellPrice = 0

    return pd.DataFrame({'Long':LongTrade, 'Short':ShortTrade}, index=ohlc.index),\
            pd.DataFrame({'Long':LongPL, 'Short':ShortPL}, index=ohlc.index)

Trade, PL = Backtest(df_codes['1311'], BuyEntry, SellEntry, BuyExit, SellExit)

In [141]:
def BacktestReport(Trade, PL):
    LongPL = PL['Long']
    LongTrades = np.count_nonzero(Trade['Long'])//2
    LongWinTrades = np.count_nonzero(LongPL.clip_lower(0))
    LongLoseTrades = np.count_nonzero(LongPL.clip_upper(0))
    print('買いトレード数 =', LongTrades)
    print('勝トレード数 =', LongWinTrades)
    print('最大勝トレード =', LongPL.max())
    print('平均勝トレード =', round(LongPL.clip_lower(0).sum()/LongWinTrades, 2))
    print('負トレード数 =', LongLoseTrades)
    print('最大負トレード =', LongPL.min())
    print('平均負トレード =', round(LongPL.clip_upper(0).sum()/LongLoseTrades, 2))
    #print('勝率 =', round(LongWinTrades/LongTrades*100, 2), '%\n')

    ShortPL = PL['Short']
    ShortTrades = np.count_nonzero(Trade['Short'])//2
    ShortWinTrades = np.count_nonzero(ShortPL.clip_lower(0))
    ShortLoseTrades = np.count_nonzero(ShortPL.clip_upper(0))
    print('売りトレード数 =', ShortTrades)
    print('勝トレード数 =', ShortWinTrades)
    print('最大勝トレード =', ShortPL.max())
    print('平均勝トレード =', round(ShortPL.clip_lower(0).sum()/ShortWinTrades, 2))
    print('負トレード数 =', ShortLoseTrades)
    print('最大負トレード =', ShortPL.min())
    print('平均負トレード =', round(ShortPL.clip_upper(0).sum()/ShortLoseTrades, 2))
    print('勝率 =', round(ShortWinTrades/ShortTrades*100, 2), '%\n')

    Trades = LongTrades + ShortTrades
    WinTrades = LongWinTrades+ShortWinTrades
    LoseTrades = LongLoseTrades+ShortLoseTrades
    print('総トレード数 =', Trades)
    print('勝トレード数 =', WinTrades)
    print('最大勝トレード =', max(LongPL.max(), ShortPL.max()))
    print('平均勝トレード =', round((LongPL.clip_lower(0).sum()+ShortPL.clip_lower(0).sum())/WinTrades, 2))
    print('負トレード数 =', LoseTrades)
    print('最大負トレード =', min(LongPL.min(), ShortPL.min()))
    print('平均負トレード =', round((LongPL.clip_upper(0).sum()+ShortPL.clip_upper(0).sum())/LoseTrades, 2))
    print('勝率 =', round(WinTrades/Trades*100, 2), '%\n')

    GrossProfit = LongPL.clip_lower(0).sum()+ShortPL.clip_lower(0).sum()
    GrossLoss = LongPL.clip_upper(0).sum()+ShortPL.clip_upper(0).sum()
    Profit = GrossProfit+GrossLoss
    Equity = (LongPL+ShortPL).cumsum()
    MDD = (Equity.cummax()-Equity).max()
    print('総利益 =', round(GrossProfit, 2))
    print('総損失 =', round(GrossLoss, 2))
    print('総損益 =', round(Profit, 2))
    print('プロフィットファクター =', round(-GrossProfit/GrossLoss, 2))
    print('平均損益 =', round(Profit/Trades, 2))
    print('最大ドローダウン =', round(MDD, 2))
    print('リカバリーファクター =', round(Profit/MDD, 2))
    return Equity

Equity = BacktestReport(Trade, PL)

買いトレード数 = 0
勝トレード数 = 0
最大勝トレード = 0.0
平均勝トレード = nan
負トレード数 = 0
最大負トレード = 0.0
平均負トレード = nan
売りトレード数 = 1
勝トレード数 = 0
最大勝トレード = 0.0
平均勝トレード = nan
負トレード数 = 1
最大負トレード = -900.0
平均負トレード = -900.0
勝率 = 0.0 %

総トレード数 = 1
勝トレード数 = 0


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars


TypeError: 'numpy.float64' object is not callable